In [1]:
!pip install -U scikit-learn

   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   --- ------------------------------------ 1.0/10.6 MB 20.5 MB/s eta 0:00:01
   ------ --------------------------------- 1.6/10.6 MB 17.3 MB/s eta 0:00:01
   ------- -------------------------------- 1.9/10.6 MB 13.5 MB/s eta 0:00:01
   -------- ------------------------------- 2.2/10.6 MB 11.5 MB/s eta 0:00:01
   -------- ------------------------------- 2.3/10.6 MB 11.2 MB/s eta 0:00:01
   -------- ------------------------------- 2.3/10.6 MB 11.2 MB/s eta 0:00:01
   -------- ------------------------------- 2.3/10.6 MB 11.2 MB/s eta 0:00:01
   -------- ------------------------------- 2.3/10.6 MB 11.2 MB/s eta 0:00:01
   -------- ------------------------------- 2.3/10.6 MB 5.5 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/10.6 MB 6.1 MB/s eta 0:00:02
   -------------- ------------------------- 3.9/10.6 MB 7.5 MB/s eta 0:00:01
   --------------- ------------------------ 4.2/10.6 MB 7.5 MB/s eta 0:00:0


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import math
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as numpy 

In [7]:
#Data = pd.read_csv("xor.csv")
#Data.head()

Data= pd.dataframe

KeyError: "None of [Index(['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous'], dtype='object')] are in the [columns]"

In [43]:
data_train, data_test = train_test_split(Data, test_size = 0.5, random_state = 4)
#data_train = Data
#data_test = Data
x_train = data_train.drop("y", axis = 1)
y_train = data_train["y"]
x_test = data_test.drop("y", axis = 1)
y_test = data_test["y"]
#x_train.head()

scaler= StandardScaler()
scaler.fit(x_train)
x_train= scaler.transform(x_train)
x_test= scaler.transform(x_test)

In [42]:
#K_cent = 4
K_cent = 8
km = KMeans(n_clusters = K_cent, max_iter= 100)
km.fit(x_train)
cent = km.cluster_centers_

In [27]:
max = 0
for i in range(K_cent):
    for j in range(K_cent):
        d = numpy.linalg.norm(cent[i] - cent[j])
        if d > max:
            max = d

d = max
sigma = d / math.sqrt(2 * K_cent)
print(sigma)

2.3008039704690626


In [44]:
#x_train = x_train.to_numpy()
#print(x_train)

In [28]:
shape = x_train.shape
row = shape[0]
column = K_cent
G = numpy.empty((row, column), dtype = float)
for i in range(row):
    for j in range(column):
        dist = numpy.linalg.norm(x_train[i] - cent[j])
        G[i][j] = math.exp(-math.pow(dist, 2) / math.pow(2 * sigma, 2))
print(G)

[[0.20436256 0.20819233 0.65386682 ... 0.06824571 0.25146433 0.17443489]
 [0.82042934 0.46448348 0.31728335 ... 0.40036471 0.60208633 0.46462202]
 [0.65388819 0.50952988 0.3606011  ... 0.19929639 0.74285211 0.47018241]
 ...
 [0.34131986 0.3585863  0.24402377 ... 0.13919446 0.4512397  0.59540017]
 [0.47298751 0.69234573 0.28991046 ... 0.16993752 0.54943978 0.43520219]
 [0.7911709  0.50446032 0.3308206  ... 0.27781562 0.61145554 0.47271694]]


In [29]:
GTG = numpy.dot(G.T, G)
GTG_inv = numpy.linalg.inv(GTG)
fac = numpy.dot(GTG_inv, G.T)
W = numpy.dot(fac, y_train)
print(W)

[ 0.37048369 -0.14348252  0.27088798  0.22065522  0.08961704 -0.23122891
 -0.12235456 -0.0801078 ]


In [37]:
row = x_test.shape[0]
column = K_cent
G_test = numpy.empty((row, column), dtype = float)
for i in range(row):
    for j in range(column):
        #dist = numpy.linalg.norm(x_test.iloc[i] - cent[j])
        dist = numpy.linalg.norm(x_test[i] - cent[j])
        G_test[i][j] = math.exp(-math.pow(dist, 2) / math.pow(2 * sigma, 2))
print(G_test[0])

[0.73694632 0.46976347 0.30593547 0.17656931 0.04743705 0.34945325
 0.58466092 0.45993041]


In [41]:
prediction = numpy.dot(G_test, W)
prediction = 0.5 * (numpy.sign(prediction - 0.5) + 1)
score = accuracy_score(prediction, y_test)
print(score.mean())

0.8860479518711847
